<a href="https://colab.research.google.com/github/YolandaMDavis/NSSADNN_IQA/blob/wildtrack-iqa/wildtrack_multitask_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Only needed to copy data to local drive can be skipped if zip file is already available in working folder
import shutil
from zipfile import ZipFile

# mount google drive
from google.colab import drive
drive.mount('/content/drive') # for google colab. adjust accordingly
PARENT_DIR = '/content/drive/MyDrive/Wildtrack Group/IQA' 

# copy and extract tar file
shutil.copy(PARENT_DIR + '/data/WildTrack_Raw.zip', 'WildTrack_Raw.zip')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'WildTrack_Raw.zip'

In [ ]:
import os

# Clone repo and copy in images. change working directory to repo's wildtrack branch
!git clone https://github.com/YolandaMDavis/NSSADNN_IQA.git
%cd "NSSADNN_IQA"
!git checkout wildtrack-iqa

# copy data files from share  if needed, comment out if already in working directory
shutil.copy( PARENT_DIR + '/data/training_image_references.csv','training_image_references.csv') 
shutil.copy( PARENT_DIR + '/data/validation_image_references.csv','validation_image_references.csv') 
shutil.copy( PARENT_DIR + '/data/testing_image_references.csv','testing_image_references.csv') 


Cloning into 'NSSADNN_IQA'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 99 (delta 46), reused 42 (delta 12), pack-reused 0
Unpacking objects: 100% (99/99), done.
/content/NSSADNN_IQA
Branch 'wildtrack-iqa' set up to track remote branch 'wildtrack-iqa' from 'origin'.
Switched to a new branch 'wildtrack-iqa'


'testing_image_references.csv'

In [ ]:
# directory variables
#%cd "NSSADNN_IQA"
import os
root_dir = '/content/NSSADNN_IQA'
data_dir = root_dir + '/RAW'
image_reference_file_suffix = '_image_references.csv'

In [ ]:
import shutil
import numpy as np
from scipy import stats
import torch
import torch.nn as nn
import random
from network import NSSADNN
from WildTrackDataset import WildTrackDataset


In [ ]:

save_model = root_dir + "/model.pth"

seed = random.randint(10000000, 99999999)

torch.manual_seed(seed)
np.random.seed(seed)
print("seed:", seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

#device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

binary_threshold = 4
train_dataset = WildTrackDataset(root_dir + '/' + 'training' + image_reference_file_suffix, binary_threshold)
val_dataset = WildTrackDataset(root_dir + '/' + 'validation' + image_reference_file_suffix, binary_threshold)
test_dataset = WildTrackDataset(root_dir + '/' + 'testing' + image_reference_file_suffix, binary_threshold)

seed: 91798733


/content/NSSADNN_IQA/WildTrackDataset.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  image_features['species'] = images['species'].apply(lambda x: self.categories[x])


In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
batch_size = 16
epochs = 10000
lr = 0.000001

train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True,
                                            pin_memory=True,
                                            num_workers=0,  drop_last=True )

val_loader = torch.utils.data.DataLoader(val_dataset)
valnum = len(val_dataset.label)


test_loader = torch.utils.data.DataLoader(test_dataset)
testnum = len(test_dataset.label)

model = NSSADNN().to(device)
classify_criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
best_SROCC = -1

# training 
for epoch in range(epochs):
      # train
      model.train()
      LOSS_all = 0
      for i, (label, features) in enumerate(train_loader):
          label = label.type(torch.FloatTensor) 
          label = label.to(device).reshape(-1,1)
          features = features.to(device).float()
          optimizer.zero_grad()
          results = model(features)         
          loss = classify_criterion(results, label)
          loss.backward()
          optimizer.step()
          LOSS_all += float(loss.item())
          
      train_loss_all = LOSS_all / (i + 1)
      
      print("Epoc {},Quality Loss={:.3f}".format(epoch,train_loss_all))
      
      # val
      y_pred = np.zeros(valnum)
      y_val = np.zeros(valnum)
      model.eval()
      L = 0
      with torch.no_grad():
          for i, (label, features) in enumerate(val_loader):
              label = label.type(torch.FloatTensor) 
              label = label.to(device).reshape(-1,1)
              y_val[i] = label.item()
              features = features.to(device).float()
              results = model(features)
              y_pred[i] = (results > .5).float() * 1 
              loss = classify_criterion(results, label)
              L = L + loss.item()
      
          val_loss = L / (i + 1)
          val_SROCC = stats.spearmanr(y_pred, y_val)[0]
          val_PLCC = stats.pearsonr(y_pred, y_val)[0]
          val_KROCC = stats.stats.kendalltau(y_pred, y_val)[0]
          val_RMSE = np.sqrt(((y_pred - y_val) ** 2).mean())

           
      print("Epoch {} Valid Results: loss={:.3f} SROCC={:.3f} PLCC={:.3f} KROCC={:.3f} RMSE={:.3f}".format(epoch,
                                                                                                            val_loss,
                                                                                                            val_SROCC,
                                                                                                            val_PLCC,
                                                                                                            val_KROCC,
                                                                                                            val_RMSE))

      if val_SROCC > best_SROCC and epoch > 100:
          print("Update Epoch {} best valid SROCC".format(epoch))
          print("Valid Results: loss={:.3f} SROCC={:.3f} PLCC={:.3f} KROCC={:.3f} RMSE={:.3f}".format(val_loss,
                                                                                                      val_SROCC,
                                                                                                      val_PLCC,
                                                                                                      val_KROCC,
                                                                                                      val_RMSE))

          torch.save(model.state_dict(), save_model)
          shutil.copy(save_model, PARENT_DIR + '/data/model.pth')
          best_SROCC = val_SROCC


Streaming output truncated to the last 5000 lines.
Epoc 7501,Quality Loss=0.601
Epoch 7501 Valid Results: loss=1.036 SROCC=0.372 PLCC=0.372 KROCC=0.372 RMSE=0.572
Epoc 7502,Quality Loss=0.601
Epoch 7502 Valid Results: loss=0.941 SROCC=0.396 PLCC=0.396 KROCC=0.396 RMSE=0.569
Epoc 7503,Quality Loss=0.606
Epoch 7503 Valid Results: loss=0.737 SROCC=0.352 PLCC=0.352 KROCC=0.352 RMSE=0.586
Epoc 7504,Quality Loss=0.603
Epoch 7504 Valid Results: loss=1.215 SROCC=0.331 PLCC=0.331 KROCC=0.331 RMSE=0.582
Epoc 7505,Quality Loss=0.607
Epoch 7505 Valid Results: loss=0.934 SROCC=0.384 PLCC=0.384 KROCC=0.384 RMSE=0.570
Epoc 7506,Quality Loss=0.607
Epoch 7506 Valid Results: loss=0.827 SROCC=0.390 PLCC=0.390 KROCC=0.390 RMSE=0.565
Epoc 7507,Quality Loss=0.602
Epoch 7507 Valid Results: loss=0.942 SROCC=0.367 PLCC=0.367 KROCC=0.367 RMSE=0.573
Epoc 7508,Quality Loss=0.611
Epoch 7508 Valid Results: loss=1.212 SROCC=0.405 PLCC=0.405 KROCC=0.405 RMSE=0.558
Epoc 7509,Quality Loss=0.608
Epoch 7509 Valid Results

In [ ]:
shutil.copy(PARENT_DIR + '/data/model.pth', save_model)
model.load_state_dict(torch.load(save_model))

model.eval()
with torch.no_grad():
    y_pred = np.zeros(testnum)
    y_test = np.zeros(testnum)
    L = 0
    for i, ( label, features) in enumerate(test_loader):
        y_test[i] = label.item()
        label = label.to(device)
        features = features.to(device).float()
        results = model(features)
        y_pred[i] = (results > .5).float() * 1 
        loss = classify_criterion(results, label)
        L = L + loss.item()
test_loss = L / (i + 1)
SROCC = stats.spearmanr(y_pred, y_test)[0]
PLCC = stats.pearsonr(y_pred, y_test)[0]
KROCC = stats.stats.kendalltau(y_pred, y_test)[0]
RMSE = np.sqrt(((y_pred - y_test) ** 2).mean())

print(y_test)
print(y_pred)

print("Final test Results: loss={:.3f} SROCC={:.3f} PLCC={:.3f} KROCC={:.3f} RMSE={:.3f}".format(test_loss, SROCC, PLCC, KROCC, RMSE))


[0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1.
 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1.
 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1.
 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1.

In [ ]:
from sklearn.metrics import precision_score, recall_score, balanced_accuracy_score, roc_auc_score
print(balanced_accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))
print(roc_auc_score(y_test, y_pred))

0.6824935189421171
0.7711442786069652
0.5794392523364486
0.682493518942117
